In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
train = pd.read_csv('../Datasets/SPECT Heart/spect_train_binary.csv')
test = pd.read_csv('../Datasets/SPECT Heart/spect_test_binary.csv')

In [3]:
X_train = train.drop('OVERALL_DIAGNOSIS', axis=1)
y_train = train['OVERALL_DIAGNOSIS']
X_test = test.drop('OVERALL_DIAGNOSIS', axis=1)
y_test = test['OVERALL_DIAGNOSIS']


# AUTOML

In [4]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120)
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

KeyboardInterrupt: 

In [ ]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8128342245989305

In [ ]:
import pickle
with open('./models/spect_heart_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [ ]:
import pickle
with open('./models/spect_heart_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

automl.show_models()

# RANDOM FOREST

In [12]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [13]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7647058823529411

# SVC

In [13]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['linear', 'ploy', 'rbf', 'sigmoid'], 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    n

GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'ploy', 'rbf', 'sigmoid']},
             verbose=3)

In [14]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8449197860962567

In [11]:
import pickle
with open('./models/spect_heart_gridsearch.pkl', 'wb') as f:
    pickle.dump(gridSearch, f)

# XGBOOST

In [20]:
import xgboost

xgb = xgboost.cv({'n_estimators': 1000, 'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'eval_metric': 'error'}, xgboost.DMatrix(X_train, y_train), num_boost_round=100, early_stopping_rounds=10, nfold=5)

[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or som

In [21]:
xgb

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.187500,0.026146,0.4000,0.115920
1,0.184375,0.026882,0.3375,0.030619
2,0.187500,0.032775,0.3375,0.050000
3,0.178125,0.028980,0.3000,0.100000
4,0.168750,0.030298,0.2875,0.101550
5,0.175000,0.033366,0.2750,0.101550
6,0.171875,0.035630,0.2750,0.101550
7,0.168750,0.030298,0.2625,0.082916
8,0.171875,0.035630,0.2750,0.101550
9,0.159375,0.025000,0.2375,0.091856


In [23]:
model = xgboost.XGBClassifier(n_estimators=1000, max_depth=6, eta=0.1, silent=1, objective='binary:logistic', eval_metric='error')

In [24]:
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

[14:57:16] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.679144385026738

# LOAD MODEL

In [4]:
import pickle
with open('./models/spect_heart_gridsearch.pkl', 'rb') as f:
    model = pickle.load(f)

/home/krish/anaconda3/envs/interpret_final/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/krish/anaconda3/envs/interpret_final/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# LIME

In [5]:
import lime
import lime.lime_tabular
import tqdm 

In [6]:
categorical_features = X_train.columns.tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['Normal', 'Abnormal'], categorical_features=categorical_features, discretize_continuous=True)

# LIME Global

In [8]:
from lime import submodular_pick
import time

start_time = time.time()
exp1 = submodular_pick.SubmodularPick(explainer, X_test.values, model.predict_proba, sample_size=200, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
exp2 = submodular_pick.SubmodularPick(explainer, X_test.values, model.predict_proba, sample_size=200, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

/home/krish/anaconda3/envs/interpret_final/lib/python3.7/site-packages/lime/submodular_pick.py:58: UserWarning: Requested sample size larger than
                              size of input data. Using all data
  size of input data. Using all data""")


--- 492.9522805213928 seconds ---


/home/krish/anaconda3/envs/interpret_final/lib/python3.7/site-packages/lime/submodular_pick.py:58: UserWarning: Requested sample size larger than
                              size of input data. Using all data
  size of input data. Using all data""")


--- 387.9769353866577 seconds ---


In [9]:
def get_feature_imp(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff[0].values

In [10]:
feat_imp1 = get_feature_imp(exp1)
feat_imp2 = get_feature_imp(exp2)

In [11]:
feat_imp2.shape, feat_imp1.shape

((44,), (44,))

In [12]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

In [13]:
i = global_identity(feat_imp1, feat_imp2)
i

0.11363636363636363

In [14]:
def normal_fi(feat_imp):
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [15]:
normal_feat_imp = normal_fi(feat_imp1 + 1e-9)

In [16]:
normal_fi(feat_imp1 + 1e-9)

array([7.88736765e-02, 1.84596917e-09, 1.84596917e-09, 5.54562925e-02,
       2.12645588e-03, 5.14438594e-02, 1.47592669e-03, 5.01377908e-02,
       1.47207248e-03, 7.60545845e-03, 5.58484580e-03, 3.84611421e-02,
       3.96035028e-02, 4.45933647e-03, 7.85915358e-03, 2.22457175e-03,
       2.69377425e-03, 4.94490665e-03, 3.07274137e-02, 5.90898539e-03,
       4.29882851e-03, 6.95230346e-04, 2.66174618e-03, 6.75105189e-02,
       1.84596917e-09, 1.84596917e-09, 4.98234617e-02, 3.91251627e-02,
       1.84596917e-09, 3.56175051e-02, 3.54712521e-02, 2.95123187e-02,
       1.22682646e-03, 3.05414481e-02, 1.84596917e-09, 2.04725307e-02,
       7.59508405e-02, 5.56806453e-02, 3.70541297e-02, 2.13004458e-02,
       1.88741508e-02, 4.60873093e-02, 3.70364711e-02, 1.84596917e-09])

In [17]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp*np.log(normal_feat_imp))/np.log(1/len(normal_feat_imp))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp*np.log(normal_feat_imp/(1/len(normal_feat_imp))))

In [18]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [19]:
Sg = calc_gini(normal_feat_imp)

In [20]:
Ser, Skl, Sg

(0.847706162954993, 0.5763087594553525, 0.0002884150378007444)

In [21]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [22]:
calc_alpha_fi(normal_feat_imp, 0.8)

0.8181818181818181

In [23]:
def get_feature_imp_all(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff

In [24]:
class1_feat_imp, class2_feat_imp = get_feature_imp_all(exp1)[0].values, get_feature_imp_all(exp1)[1].values
normal_class1_fi, normal_class2_fi = normal_fi(class1_feat_imp), normal_fi(class2_feat_imp)

In [25]:
np.linalg.norm(normal_class1_fi - normal_class2_fi, ord=2)

0.11749831283280664

# CIU

In [6]:
from ciu import determine_ciu
import tqdm
import metrics

In [7]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [8]:
feat_list = X_train.columns.tolist()

In [9]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], model.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [10]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 187/187 [02:51<00:00,  1.09it/s]


In [19]:
np.save('./explanations/spect_heart_ciu1.npy', exp1)
np.save('./explanations/spect_heart_ciu2.npy', exp2)

In [12]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [13]:
i, s, sb

((100.0, 0, 187), (0, 187, 34969, 0.0), (21, 187))

In [14]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [15]:
sim

0.0038507438287329184

In [16]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(model, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 187/187 [00:15<00:00, 12.14it/s]


In [17]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.22007902420046838
0.0
0.0
0.15979672501411638
0.0
0.0


In [18]:
metrics.calc_trust_score(model, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 187/187 [00:06<00:00, 27.76it/s]


0.25133689839572193

# RULEFIT

In [4]:
from skrules import SkopeRules
import metrics_rules
import time

In [5]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [6]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 12.452015161514282 seconds ---


In [7]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08147931098937988 seconds ---


In [8]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 187, 187)
(11358, 187, 34969, 32.48019674568904)
(68, 187)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [9]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.0


# RULEMATRIX

In [6]:
categorical_features = X_train.columns.tolist()
continuous_features = X_train.columns.drop(categorical_features).tolist()

In [7]:
import rulematrix
import time
import metrics_rules

In [8]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [9]:
surrogate = rulematrix.surrogate.rule_surrogate(
    model.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [10]:
test_x = X_test.to_numpy()

In [11]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [12]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.025736331939697266 seconds ---


In [13]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [14]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [15]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 187, 187)
(5676, 187, 34969, 16.231519345706197)
(90, 187)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [16]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [17]:
sim

0.0

# ANCHOR Global


In [5]:
from anchor import anchor_tabular
import anchor_utils
import tqdm

In [6]:
explainer = anchor_tabular.AnchorTabularExplainer(
    y_train.unique().tolist(),
    X_train.columns.tolist(),
    X_train.values
)

In [7]:
# Feature Importance using Anchor
def calc_fi(X_test, model, explainer):
    all_exps = []
    for i in tqdm.tqdm(range(len(X_test))):
        exp = explainer.explain_instance(X_test.values[i], model.predict, threshold=0.95)
        all_exps.append(exp.exp_map)
    fi = anchor_utils.greedy_pick_anchor(all_exps, X_test.values, k = len(X_test.columns))
    return fi
        

In [9]:
exp1 = calc_fi(X_test, model, explainer)
exp2 = calc_fi(X_test, model, explainer)

100%|██████████| 187/187 [01:13<00:00,  2.54it/s]


0 0.2727272727272727
1 0.5401069518716578
2 0.7058823529411765
3 0.7700534759358288
4 0.8235294117647058
5 0.8609625668449198
6 0.8877005347593583
7 0.9144385026737968
8 0.9358288770053476
9 0.9572192513368984
10 0.9679144385026738
11 0.9786096256684492
12 0.9893048128342246
13 0.9946524064171123
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0
20 1.0
21 1.0


100%|██████████| 187/187 [01:08<00:00,  2.72it/s]

0 0.2887700534759358
1 0.5240641711229946
2 0.6524064171122995
3 0.7272727272727273
4 0.7754010695187166
5 0.8128342245989305
6 0.8449197860962567
7 0.8716577540106952
8 0.8877005347593583
9 0.9037433155080213
10 0.9197860962566845
11 0.93048128342246
12 0.9411764705882353
13 0.9518716577540107
14 0.9625668449197861
15 0.9679144385026738
16 0.9732620320855615
17 0.9786096256684492
18 0.983957219251337
19 0.9893048128342246
20 0.9946524064171123
21 1.0


In [10]:
def normal_fi(feat_imp):
    feat_imp = np.array(feat_imp) + 1e-9
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [11]:
normal_feat_imp1 = normal_fi(exp1)
normal_feat_imp2 = normal_fi(exp2)

In [12]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

i = global_identity(normal_feat_imp1, normal_feat_imp2)
i


0.0

In [13]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp1*np.log(normal_feat_imp1))/np.log(1/len(normal_feat_imp1))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp1*np.log(normal_feat_imp1/(1/len(normal_feat_imp1))))

In [14]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [15]:
Sg = calc_gini(normal_feat_imp1)

In [16]:
Ser, Skl, Sg

(0.8257566224253843, 0.538593677299679, 0.0010718581461060955)

In [17]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [18]:
calc_alpha_fi(normal_feat_imp1, 0.8)

0.8181818181818181